# **Library Import**

In [ ]:
!pip install transformers

In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf 
from transformers import BertTokenizer

from copy import deepcopy

from typing import Union, List

from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer

from copy import deepcopy
import numpy as np
import csv
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from tensorflow.keras.utils import to_categorical

import transformers
from transformers import AutoTokenizer,TFBertModel


import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **Data Preprocessing**

In [3]:
dataset = pd.read_csv (r'dataset.csv',names=['description'])
classes = []
labels=[]
catagories = ["__label__bug","__label__enhancement","__label__question"]

for line in range(0,len(dataset)):
    class_name = dataset.iloc[line].to_string(header=False, index=False).split(" ")[0]
    if class_name  in catagories:
      if class_name == "__label__bug":
        labels.append(0)
      elif class_name == "__label__enhancement":
        labels.append(1)
      else:
         labels.append(2)
      dataset.iloc[line] = dataset.iloc[line].str.replace(class_name,'')
    else:
      labels.append(pd.NaT)
      dataset.iloc[line] = pd.NaT

dataset['tag'] = labels


encoded_dict = {"__label__bug":0,"__label__enhancement":1,"__label__question":2}


In [4]:
# Dropping null rows and splitting the data into test and train dataset
dataset.dropna(axis = 0, how = 'all', inplace = True)
dataset = dataset.reset_index()
train_size = int(len(dataset)*0.8)
df_train = dataset[:train_size]
df_test = dataset[train_size+1:]
print("Train dataset length: ", len(df_train), " Test dataset length: ", len(df_test))

Train dataset length:  27232  Test dataset length:  6808


In [5]:
y_train = to_categorical(df_train.tag)
y_test = to_categorical(df_test.tag)

In [6]:
#tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [7]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=df_train.description.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=df_test.description.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

# **Model Building**

In [8]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [9]:
max_len = 70
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(3,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [10]:
#Model compilation - Defining learning parameters and compiling the model.

optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [21]:
# Model Training

train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=4,
    batch_size=36
)

Epoch 1/4
757/757 [==============================] - 819s 1s/step - loss: 0.4991 - balanced_accuracy: 0.8074 - val_loss: 0.5423 - val_balanced_accuracy: 0.7920
Epoch 2/4
757/757 [==============================] - 816s 1s/step - loss: 0.4653 - balanced_accuracy: 0.8209 - val_loss: 0.5407 - val_balanced_accuracy: 0.7932
Epoch 3/4
757/757 [==============================] - 817s 1s/step - loss: 0.4396 - balanced_accuracy: 0.8313 - val_loss: 0.5455 - val_balanced_accuracy: 0.7908
Epoch 4/4
757/757 [==============================] - 819s 1s/step - loss: 0.4188 - balanced_accuracy: 0.8411 - val_loss: 0.5570 - val_balanced_accuracy: 0.7925


# **Model Evaluation**

In [12]:
# Testing our model on the test data.

predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

array([0.36498392, 0.3274018 , 0.6553406 ], dtype=float32)

In [27]:
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true =np.argmax(y_test,axis=1) 


print(len(y_predicted), len(y_true))


6808 6808


# **Classification Report** 

In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83      3498
           1       0.78      0.81      0.79      2589
           2       0.67      0.41      0.50       721

    accuracy                           0.79      6808
   macro avg       0.75      0.69      0.71      6808
weighted avg       0.78      0.79      0.78      6808



# **Infereance Prediction Pipeline**

In [29]:
texts = input(str('input the text'))
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

input the textResolve this
__label__bug 70.87471
__label__enhancement 58.55485
__label__question 26.552471
